In [1]:
import pandas as pd
import numpy as np

In [2]:
#Extraction des data
def get_data (url):
    data = pd.read_csv(url,sep="\t", header=None)
    data.columns = ["id", "label", "text"]
    return data

train_data = get_data('twitter-2013train-A.txt')
dev_data = get_data('twitter-2013dev-A.txt')
test_data = get_data('twitter-2013test-A.txt')


In [3]:
#2) Mise en place du lexique du corpus
#nous supprimons toutes les ponctuations
# avec string.punctuation: !"#$%&'()*+, -./:;?@[\]^_`{|}~
import string
def remove_punctuation(text):
    translator = str.maketrans('','', string.punctuation)
    return text.translate(translator)

#nous supprimons les espaces en trop dans un tweet
def remove_whitespace(text):
    return " ".join(text.split())

#tokenisation
from nltk.tokenize import word_tokenize

def tokenization(text):
    tokens = word_tokenize(text)
    return tokens

#Amelioration
#Supprimons la casse
def text_lowercase(text):
    return text.lower()

#Stemming: Ne garder que la racine des mots
from nltk.stem.porter import PorterStemmer
def stem_words(tokens):
    stemmer = PorterStemmer()
    stems = [stemmer.stem(word) for word in tokens]
    return stems

#Suppression des mots outils, des mots qui n'apporte pas d'information donc on peut les retirer sans affecter le sens de la phrase
from nltk.corpus import stopwords

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'));
    return [token for token in tokens if token not in stop_words]
#construction du lexique

def create_lexique(data):
    lexique = []
    for tweet in data['text']:
        lower_tweet = text_lowercase(tweet)
        tweet_without_punctuation = remove_punctuation(lower_tweet)
        new_tweet = remove_whitespace(lower_tweet)
        tokens = tokenization(new_tweet)
        tokens_with_stemming = stem_words(tokens)
        tokens_wthout_stpWords = remove_stopwords(tokens_with_stemming)
        for word in tokens:
            if word not in lexique:
                lexique.append(word)
    return lexique
    
train_data_lexique = create_lexique(train_data)


In [4]:
#Assignation d'un numero unique a chaque lexique

def assign_id_to_lexique(lexique):
    new_lexique = {}
    for i in range(1,len(lexique)+1):
        new_lexique[lexique[i-1]] = i
    return new_lexique

train_lexique_with_id = assign_id_to_lexique(train_data_lexique)
len(train_lexique_with_id)


27486

In [ ]:
#4) Decomptage pour chaque message, le nombre d'occurence des mots
# [0 for i in range(len(train_data['text'])]


def words_occurence(data, data_lexique_with_id):
    tab = []
    for tweet in data['text']:
        vector = [0 for i in range(len(data_lexique_with_id)+1)]
        for word in tokenization(tweet):
            if word in data_lexique_with_id.keys():
                vector[data_lexique_with_id[word]] +=1
               
        tab.append(vector)
    return np.array(tab)

train_tab = words_occurence(train_data, train_lexique_with_id)
dev_tab = words_occurence(dev_data, train_lexique_with_id)
test_tab = words_occurence(test_data, train_lexique_with_id)




In [8]:
test_tab.shape

(3547, 27487)

In [9]:
#5) construction du fichier au format supporte par le SVM

def generate_svm_file(data, tab, file):
    dict = {}
    label_vector = []
    for label in data['label']:
        if label == 'positive':
            label = 1
        elif label == 'negative':
            label = 2
        elif label == 'neutral':
            label = 3
        label_vector.append(label)
    #train_svm = open("train.svm", "a")
    #for tweet in train_data['text']:
    train_data
    label_vector = np.array(label_vector)
    #print(np.array(label_vector))
    # il faut construire une structure qui contient, pour chauque ligne,le label, le mot avec sa clef et l'occurence

    # la j'ajoute a chaque ligne le label correspondant au tweet
    new_vector = np.insert(tab, 0, label_vector, axis=1)
    #print(new_vector)
    fichier = open(file, "a")
    for tweet in new_vector:
        label = str(tweet[0])
        s = label
        for i in range(0, len(tweet)-1):
            c=i+1
            var = str(tweet[c])
            if tweet[c] != 0:
                s = s + ' ' + str(c) + ':' + var
        s += '\n'
        fichier.write(s)
    fichier.close()

In [10]:
generate_svm_file(train_data, train_tab, "train_svm.svm")
generate_svm_file(dev_data, dev_tab, "dev_svm.svm")
generate_svm_file(test_data, test_tab, "test_svm.svm")

In [ ]:
#Accuracy : 59.4587%
